# RUN PARCELS FOR MWB GULF STREAM SIMULATION

Alli Ho 08/24/2021

Using Jerry's code `parcels_regions.ipynb`

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import time
import numpy as np
import xarray as xr

from pathlib import Path

from src.parcels_utils import xr_dataset_to_fieldset, HFRGrid, read_netcdf_info
from src.constants import *

from src.parcels_sim import ParcelsSimulation
from src.parcels_analysis import *
import src.utils as utils

# SET UP VARIABLES

Make sure current file name is correct, otherwise only need to change the initial drop time `tstart` and the number of timesteps you want to run the simulation `repeat_N` (default is once, for N>1 the interval is determined by `repeat_dt`)

In [ ]:
currentname = 'current_netcdfs/hycom_mwbproj.nc'
projname = 'mwb_trajectories'
# lon0 = -81
# lat0 = 24.25
configname = "mwb.json"
spawnpointfile = 'spawn_points_mats/seeds_keywest.mat'
tend = 'END'
repeat_dt = 10800
simulation_dt = 300
snapshot_interval = 3600


# ADJUST FOR DESIRED FORECAST:
tstart = '2022-03-29T03:00'
repeat_N = 2

# Generate config file

## RUN PARCELS

You can make extra adjustments to the configuration in this cell if needed.

In [ ]:
configs = [
    configname
]

loaded_configs = [utils.load_config(utils.PARCELS_CONFIGS_DIR / path) for path in configs]
# modify config based on settings in the notebook
loaded_configs[0]["name"] = projname
loaded_configs[0]["netcdf_data"]["ocean"]["path"] = currentname
loaded_configs[0]["parcels_config"]["spawn_points"] = {"path": spawnpointfile}
loaded_configs[0]["parcels_config"]["time_range"] = [tstart, tend]
loaded_configs[0]["parcels_config"]["repeat_dt"] = repeat_dt
loaded_configs[0]["parcels_config"]["simulation_dt"] = simulation_dt
loaded_configs[0]["parcels_config"]["snapshot_interval"] = snapshot_interval
loaded_configs[0]["parcels_config"]["repetitions"] = repeat_N


grids = []
for cfg in loaded_configs:
    ds = read_netcdf_info(cfg["netcdf_data"]["ocean"])
    grids.append(HFRGrid(ds))
sims = []
for i in range(len(loaded_configs)):
    sim = ParcelsSimulation(loaded_configs[i]["name"], grids[i], loaded_configs[i]["parcels_config"])
    sims.append(sim)

In [ ]:
[sim.pset.show(field="vector") for sim in sims]

# RUN SIMULATION

In [ ]:
for sim in sims:
    sim.execute()